# Elo Merchant

## Introduction
This notebook represents Sean Ng's submission to elo merchant. 

I got some feature engineering ideas from:

In [1]:
#Declaring imports
import csv
import os
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
from sklearn.cross_validation import KFold
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import xgboost as xgd


/anaconda/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/envs/tensorflow/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/anaconda/envs/tensorflow/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
EXPLORE = True

In [35]:
#Files
train_data_path = '../input/train.csv'
test_data_path = '../input/test.csv'
history_path = '../input/historical_transactions.csv'
merchant_path = '../input/merchants.csv'
new_transactions_path = '../input/new_merchant_transactions.csv'


In [3]:
train_df = pd.read_csv(train_data_path)
test_df = pd.read_csv(test_data_path)

In [33]:
history_df = pd.read_csv(history_path)

In [17]:
merchants = pd.read_csv(merchant_path)

In [36]:
new_transactions_df = pd.read_csv(new_transactions_path)

,feature_1,feature_2,feature_3,target
count,201917.000000,201917.000000,201917.000000,201917.000000
mean,3.105311,1.745410,0.565569,-0.393636
std,1.186160,0.751362,0.495683,3.850500
min,1.000000,1.000000,0.000000,-33.219281
25%,2.000000,1.000000,0.000000,-0.883110
50%,3.000000,2.000000,1.000000,-0.023437
75%,4.000000,2.000000,1.000000,0.765453
max,5.000000,3.000000,1.000000,17.965068


In [27]:
all_data = train_df.append(test_df)

/anaconda/envs/tensorflow/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [53]:
all_transactions = history_df.append(new_transactions_df)

In [37]:
all_data.describe()

,feature_1,feature_2,feature_3,target
count,325540.00000,325540.000000,325540.000000,201917.000000
mean,3.10681,1.744038,0.565116,-0.393636
std,1.18728,0.750540,0.495742,3.850500
min,1.00000,1.000000,0.000000,-33.219281
25%,2.00000,1.000000,0.000000,-0.883110
50%,3.00000,2.000000,1.000000,-0.023437
75%,4.00000,2.000000,1.000000,0.765453
max,5.00000,3.000000,1.000000,17.965068


In [57]:
all_transactions.describe()

,city_id,installments,merchant_category_id,month_lag,purchase_amount,category_2,state_id,subsector_id
count,3.107539e+07,3.107539e+07,3.107539e+07,3.107539e+07,3.107539e+07,2.831078e+07,3.107539e+07,3.107539e+07
mean,1.296453e+02,6.506728e-01,4.778518e+02,-4.110560e+00,-7.032267e-04,2.194792e+00,1.058662e+01,2.679329e+01
std,1.040926e+02,2.734985e+00,2.494821e+02,3.766495e+00,1.087456e+03,1.531650e+00,6.347145e+00,9.723248e+00
min,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.300000e+01,-7.469078e-01,1.000000e+00,-1.000000e+00,-1.000000e+00
25%,5.300000e+01,0.000000e+00,3.020000e+02,-7.000000e+00,-7.199051e-01,1.000000e+00,9.000000e+00,1.900000e+01
50%,9.000000e+01,0.000000e+00,4.540000e+02,-3.000000e+00,-6.875230e-01,1.000000e+00,9.000000e+00,2.900000e+01
75%,2.120000e+02,1.000000e+00,7.050000e+02,-1.000000e+00,-6.008651e-01,3.000000e+00,1.600000e+01,3.400000e+01
max,3.470000e+02,9.990000e+02,8.910000e+02,2.000000e+00,6.010604e+06,5.000000e+00,2.400000e+01,4.100000e+01


In [56]:
print("All data shape:" + str(all_data.shape))
print("-----------------------------------------------------")
print("Train data shape:" + str(train_df.shape))
print("Test data shape:" + str(test_df.shape))
print("=====================================================")
print("All transactions data shape:" + str(all_transactions.shape))
print("-----------------------------------------------------")
print("Old transactions data shape:" + str(history.shape))
print("New transactions data shape:" + str(new_transactions_df.shape))

All data shape:(325540, 6)
-----------------------------------------------------
Train data shape:(201917, 6)
Test data shape:(123623, 5)
All transactions data shape:(31075392, 14)
-----------------------------------------------------
Old transactions data shape:(29112361, 14)
New transactions data shape:(1963031, 14)


In [77]:
def get_df_name(df):
    for x in globals():
        if globals()[x] is df:
            return name

In [75]:
#Finding number of null values
def count_nulls(df_list):
    result = {}
    for df in df_list:
        vals = {}
        for key in df.keys():
            count = df[key].isnull().sum()
            if count > 0:
                vals[key] = count
        name = get_df_name(df)
        result[name] = vals
    return pd.DataFrame.from_dict(result, dtype=int)

count_nulls([all_data, all_transactions, merchants])


,all_data,all_transactions,merchants
avg_sales_lag12,NaN,NaN,13.0
avg_sales_lag3,NaN,NaN,13.0
avg_sales_lag6,NaN,NaN,13.0
category_2,NaN,2764609.0,11887.0
category_3,NaN,234081.0,NaN
first_active_month,1.0,NaN,NaN
merchant_id,NaN,164697.0,NaN
target,123623.0,NaN,NaN


first_active_month         1
target                123623
category_3      234081
merchant_id     164697
category_2     2764609


SyntaxError: invalid syntax (<ipython-input-74-04ded49cdeb2>, line 2)

# Feature Engineering